Define interactive variables to set up particle stepper (static and variable), and then have user define figure of merit.<br>
Write out in either .csv or .txt file (idea: in .csv file, have rows correspond to objects (ie, magnets, particle beam, etc) and have the first column represent the number of those objects (magnets, beams, etc), and the remaining columns an ordered list of values defining those objects, to read from left to right, spanning all of the relevant objects).<br>
Have c++ code read this, and define objects with loops.

Items to define:<br>
1. Magenet: Center of Enterance Plane, Length, Width, Height
2. Beam: number of particles, central starting position, central energy, central direction (angles from axes)
3. Beam: spread in starting position, spread in central energy, spread in divergence (full spread)
4. Screen: Low-Energy Middle, Width, Height

![alt text](magnet_position.png "Magnet_Setup")

In [1]:
from magspecinterface_functions import *

We'll first set the units for a the quantities that the user will input into the code:

In [2]:
display( units_length )
display( units_energy )
display( units_angles )

Dropdown(description='Length Unit:', index=1, options=('mm', 'cm', 'm'), value='cm')

Dropdown(description='Energy Unit:', index=1, options=('eV', 'MeV', 'GeV'), value='MeV')

Dropdown(description='Angle Unit:', options=('mrad', 'Radians', 'Degrees'), value='mrad')

In [3]:
display(number_of_magnets)

BoundedIntText(value=2, description='# of Magnets')

In [4]:
magnet_x_position_widgets = dynamicFloatTextWidgets(number_of_magnets.value)
for ii in range(len(magnet_x_position_widgets)):
    display(magnet_x_position_widgets[ii])

FloatText(value=0.0, description='x of Mag 1')

FloatText(value=0.0, description='x of Mag 2')

In [ ]:
magnet_x_position_widgets[0].value

In [ ]:
###TEST BOXES BELOW:

In [ ]:
test = widgets.BoundedFloatText()
test.keys

4